In [1]:
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
import json
from pathlib import Path
import csv

In [2]:
class CFG():
    pretrained_model = '/larch/share/bert/NICT_pretrained_models/NICT_BERT-base_JapaneseWikipedia_32K_BPE'
    path = '/mnt/hinoki/karai/KUCI'
    max_seq_len = 128
    batch_size = 16
    lr = 2e-5
    weight_decay = 0.01
    seed = 0
    epoch = 3
    warmup_ratio = 0.033
    save_path = "./result/bert"

In [3]:
save_path = Path(CFG.save_path)
save_path.mkdir(exist_ok=True, parents=True)

In [4]:
import random
import numpy as np
import torch
import os
import re


def set_seed(seed: int) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


def set_device(gpuid: str) -> torch.device:
    if gpuid and torch.cuda.is_available():
        assert re.fullmatch(r"[0-7]", gpuid) is not None, "invalid way to specify gpuid"
        os.environ["CUDA_VISIBLE_DEVICES"] = gpuid
        device = torch.device(f"cuda:{gpuid}")
    else:
        device = torch.device("cpu")

    return device

In [5]:
set_seed(CFG.seed)
device = set_device("0")
print(device)

cuda:0


In [6]:
tokenizer = BertTokenizer.from_pretrained(
    CFG.pretrained_model, do_lower_case=False, do_basic_tokenize=False
)

print(tokenizer.tokenize('今日は曇りです'))
print(tokenizer([['今日は曇りです', '明日は晴れるといいな'],['明日はピクニックです', 'お弁当楽しみだな']]))

['今日', '##は', '##曇', '##り', '##です']
{'input_ids': [[2, 8016, 26343, 29756, 26483, 26287, 3, 15108, 26343, 29746, 26507, 26292, 25974, 26302, 3], [2, 15108, 26343, 28149, 26894, 27927, 26287, 3, 274, 29556, 29573, 29822, 26149, 26222, 26302, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


In [7]:
class PairPro(Dataset):
    def __init__(self, path, tokenizer, max_seq_len, is_test=False):
        self.is_test = is_test
        self.label2int = {'a': 0, 'b': 1, 'c': 2, 'd': 3}
        self.labels, self.contexts, self.choices = self.load(path)
        self.tokenizer = tokenizer
        self.max_seq_len = max_seq_len 
    
    def load(self, path):
        label_list = []
        context_list = []
        choice_list = []
        with open(path) as f:
            for i, line in enumerate(f):
                problem = json.loads(line)
                if self.is_test:
                    label_list.append(-1)
                else:
                    label_list.append(self.label2int[problem['label']])
                context_list.append(problem['context'])
                choice_list.append([problem['choice_a'],problem['choice_b'],problem['choice_c'],problem['choice_d']])
        assert len(label_list) == len(context_list), "長さが違います"
        return label_list, context_list, choice_list

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        input_list = []
        for choice in self.choices[idx]:
            input_list.append([self.contexts[idx], choice])    
        return self.labels[idx], self.tokenizer(
            input_list,
            max_length=self.max_seq_len, 
            truncation=True,
            padding='max_length',
            return_tensors='pt',
            )

In [8]:
traindataset = PairPro(CFG.path+'/train.jsonl', tokenizer, CFG.max_seq_len)
print(traindataset[0])  # pairの要素を参照すると__getitem__が呼び出される
devdataset = PairPro(CFG.path+'/development.jsonl', tokenizer, CFG.max_seq_len)
testdataset = PairPro(CFG.path+'/test.jsonl', tokenizer, CFG.max_seq_len, is_test=True)

(3, {'input_ids': tensor([[    2, 22845,  1135, 16968,  1159,  1484, 15060,  1143,    64,     3,
         12091,  1135,  9871, 20394, 20353,  1159, 12081,   607,  1328,  1610,
             3,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     

In [9]:
traindataloader = DataLoader(traindataset, batch_size=CFG.batch_size, shuffle=True, num_workers=2)
devataloader = DataLoader(devdataset, batch_size=CFG.batch_size, shuffle=True, num_workers=2)
testdataloader = DataLoader(testdataset, batch_size=CFG.batch_size, shuffle=False, num_workers=2)

In [10]:
class BERTPairPro(nn.Module):
    def __init__(self, pretrained_model):
        super().__init__()
        self.bert = BertModel.from_pretrained(
            pretrained_model, output_attentions=False
        )
        self.linear = nn.Linear(
            self.bert.config.hidden_size, 1
        )
    
    def forward(self, input_ids, attention_mask, token_type_ids):
        batch_size, n_choice, seq_len = input_ids.shape     # 16,4,128
        input_ids = input_ids.view(batch_size*n_choice, seq_len)        # 64,128
        attention_mask = attention_mask.view(batch_size*n_choice, seq_len)
        token_type_ids = token_type_ids.view(batch_size*n_choice, seq_len)
        # print(input_ids.shape)
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        cls = output['pooler_output']   # 64, 768
        output = self.linear(cls)   # 64, 1
        output = output.squeeze(1).view(batch_size, n_choice)
        return output

In [11]:

model = BERTPairPro(CFG.pretrained_model)
model = model.to(device)

Some weights of the model checkpoint at /larch/share/bert/NICT_pretrained_models/NICT_BERT-base_JapaneseWikipedia_32K_BPE were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
# for i, batch in enumerate(PairProdataloader):
#     #print(batch)
#     label, batch = batch
#     if i == 0:
#         output = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])
#         print(output)
#         break

In [13]:
optimizer = AdamW(
    model.parameters(),
    lr=CFG.lr,
    weight_decay=CFG.weight_decay,
    no_deprecation_warning=True
)

In [14]:
num_training_steps = len(traindataloader) * CFG.epoch
num_warmup_steps = num_training_steps * CFG.warmup_ratio
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_training_steps=num_training_steps,
    num_warmup_steps=num_warmup_steps
)

ce_loss = nn.CrossEntropyLoss()

In [15]:
best_score = None
for epoch in range(CFG.epoch):
    total_loss = 0
    model.train()
    train_bar = tqdm(traindataloader)
    for batch_idx, batch in enumerate(train_bar):
        label, batch = batch 
        batch_size = len(batch['input_ids'])
        label = label.to(device)
        batch = {key: value.to(device) for key, value in batch.items()} 
        output = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])
        loss = ce_loss(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item() * batch_size
        train_bar.set_postfix(
            {
                'lr': scheduler.get_last_lr()[0],
                'loss': round(total_loss / (batch_idx + 1), 3)
            }
        )
    

    model.eval()
    with torch.no_grad():
        dev_bar = tqdm(devataloader)
        num_correct = 0
        total_loss = 0
        size = 0 
        for batch_idx, batch in enumerate(dev_bar):
            label, batch = batch 
            batch_size = len(batch['input_ids'])
            label = label.to(device)
            batch = {key: value.to(device) for key, value in batch.items()} 
            output = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])
            loss = ce_loss(output, label)
            predictions = torch.argmax(F.softmax(output, dim=1), dim=1)
            num_correct += torch.sum(predictions == label).item()
            size += batch_size
            total_loss += loss.item() * batch_size
            score = round(num_correct/size, 3)

            dev_bar.set_postfix(
            {
                'size': size,
                'accuracy': score,
                'loss': round(total_loss / (batch_idx + 1), 3)
            }
        )
    
    score = round(num_correct/size, 3)
    if best_score is None or score > best_score:
        torch.save(model.state_dict(), CFG.save_path+"/Checkpoint_best.pth")
        best_score = score



  1%|          | 33/5196 [00:15<39:51,  2.16it/s, lr=1.28e-6, loss=21.3] 

In [ ]:
model = BERTPairPro(CFG.pretrained_model)
model = model.to(device)

Some weights of the model checkpoint at /larch/share/bert/NICT_pretrained_models/NICT_BERT-base_JapaneseWikipedia_32K_BPE were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
state_dict = torch.load(CFG.save_path+"/Checkpoint_best.pth", map_location=device)
model.load_state_dict(state_dict)   

<All keys matched successfully>

In [ ]:
prediction = []
model.eval()
with torch.no_grad():
    test_bar = tqdm(testdataloader)
    num_correct = 0
    total_loss = 0
    size = 0 
    for batch_idx, batch in enumerate(test_bar):
        label, batch = batch 
        batch_size = len(batch['input_ids'])
        # label = label.to(device)
        batch = {key: value.to(device) for key, value in batch.items()} 
        output = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])
        # loss = ce_loss(output, label)
        predictions = torch.argmax(F.softmax(output, dim=1), dim=1)
        size += batch_size
        #total_loss += loss.item() * batch_size
        prediction += predictions.tolist()
    
    int2label = {0: 'a', 1: 'b', 2: 'c', 3: 'd'}
    prediction = [int2label[pre] for pre in prediction]
    print(prediction)

    #     #dev_bar.set_postfix(
    #     {
    #         'size': size,
    #         #'accuracy': round(num_correct/size, 3),
    #         #'loss': round(total_loss / (batch_idx + 1), 3)
    #     }
    # )

    

100%|██████████| 32/32 [00:05<00:00,  6.39it/s]

['d', 'b', 'c', 'c', 'a', 'c', 'c', 'b', 'd', 'c', 'c', 'c', 'a', 'b', 'a', 'c', 'd', 'c', 'd', 'b', 'a', 'b', 'b', 'd', 'b', 'a', 'a', 'b', 'a', 'c', 'c', 'd', 'c', 'c', 'c', 'b', 'd', 'a', 'c', 'a', 'd', 'a', 'c', 'd', 'a', 'a', 'c', 'a', 'd', 'a', 'c', 'a', 'a', 'b', 'd', 'c', 'a', 'b', 'b', 'd', 'd', 'c', 'a', 'a', 'a', 'c', 'a', 'd', 'd', 'a', 'a', 'd', 'b', 'b', 'd', 'c', 'd', 'd', 'a', 'c', 'a', 'c', 'c', 'c', 'b', 'c', 'a', 'c', 'a', 'a', 'a', 'c', 'c', 'a', 'c', 'c', 'a', 'b', 'a', 'a', 'd', 'b', 'a', 'c', 'b', 'a', 'a', 'd', 'd', 'b', 'b', 'd', 'b', 'd', 'd', 'd', 'c', 'a', 'd', 'd', 'd', 'a', 'c', 'c', 'a', 'd', 'a', 'b', 'b', 'a', 'b', 'a', 'a', 'a', 'd', 'd', 'd', 'a', 'd', 'b', 'b', 'd', 'c', 'c', 'c', 'a', 'a', 'c', 'b', 'b', 'a', 'b', 'c', 'a', 'd', 'c', 'b', 'b', 'c', 'b', 'b', 'c', 'b', 'c', 'b', 'c', 'd', 'c', 'c', 'c', 'b', 'd', 'a', 'c', 'b', 'c', 'd', 'a', 'c', 'd', 'a', 'b', 'd', 'a', 'c', 'a', 'b', 'd', 'c', 'b', 'b', 'c', 'c', 'b', 'a', 'a', 'c', 'c', 'b', 'b',

In [ ]:
with open(CFG.save_path+"/test_prediction.csv", "w") as f:
  csv.writer(f).writerows(prediction)
f.close()